In [ ]:
import autograd.numpy as np
from autograd import grad
import scipy as sp
import scipy.optimize
from dataclasses import dataclass
from matplotlib import pyplot as plt

def dot(a, b):
    return np.multiply(a, b).sum(axis=-2)

def make_vec3(x, y, z):
    return np.array((x, y, z)).reshape(3, 1)

@dataclass
class SG:
    axis: np.ndarray
    amplitude: np.ndarray
    sharpness: np.ndarray

    def eval(self, v):
        r = self.amplitude * np.exp(self.sharpness * (dot(v, self.axis) - 1))
        return r.sum(axis=0)

def fit_sg(f0, X, Y, p0, lo=None, hi=None, ni=100):
    f = lambda params: f0(X, params)

    M = np.max(Y)
    loss = lambda params: np.sum(((f(params) - Y) / M) ** 2)

    c = lambda params: f0(X, params) / M

    kwargs = {
        'jac': grad(loss),
        'bounds': np.stack((lo, hi)).transpose()
    }
    res = sp.optimize.basinhopping(loss, p0, niter=ni, interval=ni//2, minimizer_kwargs=kwargs)
    
    num_negative = np.sum(f(res.x) < 0)
    if num_negative == 0:
        return res
    print(f"Reoptimize with constraints: have {num_negative}/{len(Y)} negative samples")
    kwargs['constraints'] = {
        'type': 'ineq',
        'fun': c,
        'jac': None,
    }
    return sp.optimize.basinhopping(loss, res.x, niter=ni, interval=ni//2, minimizer_kwargs=kwargs)

def polar_to_r3(phi, axis=0):
    phi = phi if len(np.shape(phi)) > 0 else np.reshape(phi, 1)
    p = np.stack((np.cos(phi), np.zeros(np.shape(phi)), np.sin(phi)), axis=axis)
    return p

def spherical_to_r3(phi, theta, axis=0):
    z = np.cos(theta)
    r = np.sqrt(1 - z * z)
    return np.stack((np.cos(phi) * r, np.sin(phi) * r, z), axis=axis)

def newton(f, x0, n):
    d_f = grad(f)
    xp = x0
    x = x0
    for i in range(n):
        x = xp - f(xp) / d_f(xp)
        xp = x
    return x

def F_schlick(f0, NoL):
    return f0 + (1 - f0) * (1 - NoL) ** 5

def G_smith(alpha, NoV, NoL):
    alpha2 = alpha ** 2;
    NoV2 = NoV ** 2
    NoL2 = NoL ** 2
    A_V = np.sqrt(1 + alpha2 * (1 - NoV2) / NoV2)
    A_L = np.sqrt(1 + alpha2 * (1 - NoL2) / NoL2)
    return 2 / (A_V + A_L)

def D_ggx(alpha, NoH):
    alpha2 = alpha ** 2
    return alpha2 / (np.pi * (1 + NoH ** 2 * (alpha2 - 1)) ** 2)

def BRDF(f0, alpha, N, V, L, mul_NoL=False):
    H = L + V
    H = H / np.linalg.norm(H, axis=0)
    HoV = dot(V, H)
    NoV = dot(N, V)
    NoL = dot(N, L)
    NoH = dot(N, H)
    F = F_schlick(f0, HoV)
    G = G_smith(alpha, NoV, NoL)
    D = D_ggx(alpha, NoH)
    if mul_NoL:
        return np.where(NoL > 0, F * G * D / (4 * NoV), 0)
    return np.where(NoL > 0, F * G * D / (4 * NoV * NoL), 0)

def r2_seq(n, seed=0.5):
    g = 1.32471795724474602596
    a1 = 1 / g
    a2 = a1 ** 2
    x, _ = np.modf(seed + a1 * n) 
    y, _ = np.modf(seed + a2 * n)
    return np.stack((x, y))

def uniform_sample_hemisphere(Xi):
    phi = 2 * np.pi * Xi[0]
    z = Xi[1]
    theta = np.acos(z)
    return (phi, theta)

def uniform_sample_sphere(Xi):
    phi = 2 * np.pi * Xi[0]
    z = 2 * Xi[1] - 1
    theta = np.acos(z)
    return (phi, theta)
        

In [ ]:
# Approximate maximum lambda value from epsilon support and cubemap texel size.

Res = 512
S = (2 / Res) ** 2
print(f"Texel area: {S}")
d = make_vec3(1, 1, 1)
d2 = 3
S = S / np.sqrt(d2)
print(f"Projected texel area: {S}")
r2 = S / np.pi
eps = 0.01
cos_theta = 1 - 0.5 * r2 / d2
print("Support angle: {}".format(np.degrees(np.acos(cos_theta))))
L_MAX = np.log(eps) / (cos_theta - 1)
print(f"Max lambda: {L_MAX}")

In [ ]:
# Fit visibility SG

@dataclass
class Cone:
    axis: np.ndarray
    aperture: np.ndarray

    def eval(self, v):
        return np.where(dot(v, self.axis) > np.cos(self.aperture), 1.0, 0.0)

def ka_from_lambda(l):
    return 2.0 * (l + np.exp(-l) - 1.0) / (l * l)

def l0_from_ka(ka):
    return 2.0 / (ka + 0.01) * (1.0 - ka) + 3.0 * (1.0 - ka) * ka

for ka in (0.1, 0.5, 0.9):
    N = make_vec3(0, 0, 1)
    
    l0 = l0_from_ka(ka)
    l1 = newton(lambda l: ka_from_lambda(l) - ka, l0, 1)
    sg = SG(np.reshape(N, (1, 3, 1)), np.reshape(1, (1, 1)), np.reshape(l1, (1, 1)))

    aperture = np.acos(np.sqrt(1.0 - ka))
    c = Cone(N, aperture)

    phi = np.linspace(0, np.pi, num=100)
    axis = polar_to_r3(phi)

    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})

    plt.title(f"Visibility fit for ka={ka}")
    ax.plot(phi, c.eval(axis), label=f"Cone")
    ax.plot(phi, sg.eval(axis), label=f"SG")
    ax.plot(phi, c.eval(axis) * np.sin(phi), label = f"Cosine-weighted cone")
    ax.plot(phi, sg.eval(axis) * np.sin(phi), label = f"Cosine-weighted SG")
    ax.set_rmax(1)
    ax.set_rticks([0.0, 0.5, 1])
    ax.grid(True)
    ax.legend()
    plt.show()

    fig, ax = plt.subplots()
    plt.title(f"Visibility fit for ka={ka}")
    ax.plot(phi - np.pi / 2.0, c.eval(axis), label=f"Cone")
    ax.plot(phi - np.pi / 2.0, sg.eval(axis), label=f"SG")
    ax.plot(phi - np.pi / 2.0, c.eval(axis) * np.sin(phi), label = f"Cosine-weighted cone")
    ax.plot(phi - np.pi / 2.0, sg.eval(axis) * np.sin(phi), label = f"Cosine-weighted SG")
    ax.set_ylim(1)
    ax.set_yticks([0.0, 0.5, 1])
    ax.grid(True)
    ax.legend()
    plt.show()

In [ ]:
# Fit normal distribution SG

for roughness in (0.1, 0.3, 0.5, 0.7, 0.9):
    N = make_vec3(0, 0, 1)
    
    alpha = roughness ** 2
    alpha2 = alpha ** 2

    phi = np.linspace(0, np.pi, num=100)
    H = polar_to_r3(phi)
    d = D_ggx(alpha, dot(N, H))

    def f(X, *args):
        args = np.array(args).reshape(-1, 2, 1)
        u = N.reshape(1, 3, 1)
        a = args[:, 0]
        l = args[:, 1]
        return SG(u, a, l).eval(X)

    sgs = []

    def fit(p0):
        k = len(p0) // 2
        lo = np.tile((-np.inf, 0), k)
        hi = np.tile(( np.inf, L_MAX), k)
        return fit_sg(f, H, d, p0, lo, hi)

    popt = []
    loss = np.inf
    for k in range(1, 3):
        i = np.argmax(np.abs(d - f(H, popt)))
        popt = np.append(popt, (0, 0))
        res = fit(popt)
        popt = res.x
        print(f"Fit {k} SG(s) in {res.nit} iteration(s): {res.message}")
        print(f"Optimal parameters:\n{popt.reshape(-1, 2)}")
        print(f"Loss: {res.fun} ({loss / res.fun}x better)")
        loss = res.fun
        sgs.append(popt)

    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    plt.title(f"D fit for Roughness={roughness}")
    ax.plot(phi, d, label="GGX")
    for i, sg in enumerate(sgs):
        k = i + 1
        print(sg)
        ax.plot(phi, f(H, sg), label=f"{k} SG(s)")
    ax.grid(True)
    ax.legend()
    plt.show()

    fig, ax = plt.subplots()
    plt.title(f"D fit for Roughness={roughness}")
    ax.plot(phi - np.pi / 2.0, d, label="GGX")
    for i, sg in enumerate(sgs):
        k = i + 1
        ax.plot(phi - np.pi / 2.0, f(H, sg), label=f"{k} SG(s)")
    ax.grid(True)
    ax.legend()
    plt.show()

In [ ]:
# Fit BRDF SG

for mul_NoL in (False, True):
    for roughness in (0.1, 0.3, 0.5, 0.7, 0.9):
        f0 = 0.04
        alpha = roughness ** 2
        alpha2 = alpha ** 2
    
        N_phi = np.pi / 2
        V_phi = 3 * np.pi / 4
        R_phi = 2 * N_phi - V_phi
        N = polar_to_r3(N_phi)
        V = polar_to_r3(V_phi)

        if mul_NoL:
            phi = np.linspace(-np.pi / 2, 3 * np.pi / 2, num=100)
        else:
            phi = np.linspace(0, np.pi, num=100)
        L = polar_to_r3(phi)
        y = BRDF(f0, alpha, N, V, L, mul_NoL=mul_NoL)
    
        def f(X, params):
            params = params.reshape(-1, 3, 1)
            u = polar_to_r3(params[:, 0], axis=1)
            a = params[:, 1]
            l = params[:, 2]
            return SG(u, a, l).eval(X)
    
        MIN_SG = 2
        MAX_SG = 4
        sgs = []
    
        def fit(p0):
            k = len(p0) // 3
            lo = np.tile((-np.pi, -np.inf, 0), k)
            hi = np.tile(( np.pi,  np.inf, L_MAX), k)
            return fit_sg(f, L, y, p0, lo, hi)
    
        res = fit(np.array([R_phi, 1 / (np.pi * alpha2), 2 / alpha2]))
        popt = res.x
        loss = res.fun
        for k in range(MIN_SG, MAX_SG + 1):
            i = np.argmax(np.abs(y - f(L, popt)))
            popt = np.append(popt, (phi[i], 0, 0))
            res = fit(popt)
            popt = res.x
            print(f"Fit {k} SG(s) in {res.nit} iteration(s): {res.message}")
            print(f"Optimal parameters:\n{popt.reshape(-1, 3)}")
            print(f"Loss: {res.fun} ({loss / res.fun}x better)")
            loss = res.fun
            sgs.append(popt)

        if mul_NoL:
            brdf_str = "BRDF * (N, L)"
            f_str = "f * (N, L)"
        else:
            brdf_str = "BRDF"
            f_str = "f"
    
        fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
        plt.title(f"{brdf_str} fit for Roughness={roughness}, f0={f0}, V_phi={V_phi}")
        ax.plot(phi, y, label=f_str)
        for k, sg in enumerate(sgs, start=MIN_SG):
            ax.plot(phi, f(L, sg), label=f"{f_str}: {k} SG(s)")
        ax.grid(True)
        ax.legend()
        plt.show()
    
        fig, ax = plt.subplots()
        plt.title(f"{brdf_str} fit for Roughness={roughness}, f0={f0}, V_phi={V_phi}")
        ax.plot(-phi + np.pi / 2, y, label=f_str)
        for k, sg in enumerate(sgs, start=MIN_SG):
            ax.plot(-phi + np.pi / 2, f(L, sg), label=f"{f_str}: {k} SG(s)")
        ax.grid(True)
        ax.legend()
        plt.show()

In [ ]:
# Fit BRDF SG in 3D

for mul_NoL in (False, True):
    for roughness in (0.1, 0.3, 0.5, 0.7, 0.9):
        f0 = 0.04
        alpha = roughness ** 2
    
        N_phi = np.pi / 2
        V_phi = 3 * np.pi / 4
        R_phi = 2 * N_phi - V_phi
        N = polar_to_r3(N_phi)
        V = polar_to_r3(V_phi)
        R = polar_to_r3(R_phi)
    
        Xi = r2_seq(np.arange(10000))
        if mul_NoL:
            (phi, theta) = uniform_sample_sphere(Xi)
        else:
            (phi, theta) = uniform_sample_hemisphere(Xi)
        L = spherical_to_r3(phi, theta)
        y = BRDF(f0, alpha, N, V, L, mul_NoL=mul_NoL)
        
        def f(X, params):
            params = params.reshape(-1, 4, 1)
            phi = params[:, 0]
            theta = params[:, 1]
            u = spherical_to_r3(phi, theta, axis=1)
            a = params[:, 2]
            l = params[:, 3]
            return SG(u, a, l).eval(X)
    
        MIN_SG = 2
        MAX_SG = 4
        sgs = []
    
        def fit(p0):
            k = len(p0) // 4
            lo = np.tile((-2 * np.pi, -np.pi, -np.inf, 0    ), k)
            hi = np.tile(( 2 * np.pi,  np.pi,  np.inf, L_MAX), k)
            return fit_sg(f, L, y, p0, lo, hi, ni=100)
    
        res = fit(np.array((np.arctan2(R[1], R[0])[0], np.acos(R[2])[0], 1 / (np.pi * alpha2), 2 / alpha2)))
        popt = res.x
        loss = res.fun
        for k in range(MIN_SG, MAX_SG + 1):
            i = np.argmax(np.abs(y - f(L, popt)))
            popt = np.append(popt, (phi[i], theta[i], 0, 0))
            res = fit(popt)
            popt = res.x
            print(f"Fit {k} SG(s) in {res.nit} iteration(s): {res.message}")
            print(f"Optimal parameters:\n{popt.reshape(-1, 4)}")
            print(f"Loss: {res.fun} ({loss / res.fun}x better)")
            loss = res.fun
            sgs.append(popt)

        if mul_NoL:
            phi = np.linspace(-np.pi / 2, 3 * np.pi / 2, num=100)
            brdf_str = "BRDF (N, L)"
            f_str = "f (N, L)"
        else:
            phi = np.linspace(0, np.pi, num=100)
            brdf_str = "BRDF"
            f_str = "f"

        L = polar_to_r3(phi)
        y = BRDF(f0, alpha, N, V, L, mul_NoL=mul_NoL)
    
        fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
        plt.title(f"{brdf_str} fit for Roughness={roughness}, f0={f0}, V_phi={V_phi}")
        ax.plot(phi, y, label=f_str)
        for k, sg in enumerate(sgs, start=MIN_SG):
            ax.plot(phi, f(L, sg), label=f"{f_str}: {k} SG(s)")
        ax.grid(True)
        ax.legend()
        plt.show()
    
        fig, ax = plt.subplots()
        plt.title(f"{brdf_str} fit for Roughness={roughness}, f0={f0}, V_phi={V_phi}")
        ax.plot(-phi + np.pi / 2, y, label=f_str)
        for k, sg in enumerate(sgs, start=MIN_SG):
            ax.plot(-phi + np.pi / 2, f(L, sg), label=f"{f_str}: {k} SG(s)")
        ax.grid(True)
        ax.legend()
        plt.show()
    
        W = 30
        H = 30
        
        Xix = np.linspace(0, 1, num=W)
        Xiy = np.linspace(0, 1, num=H)
        Xix, Xiy = np.meshgrid(Xix, Xiy)
        Xix = Xix.flatten()
        Xiy = Xiy.flatten()
        X = np.stack(uniform_sample_hemisphere((Xix, Xiy)))
        L = spherical_to_r3(*X)
        y = BRDF(f0, alpha, N, V, L, mul_NoL=mul_NoL).reshape(H, W)
        P = L.reshape(3, H, W) * y
    
        fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
        plt.title(f"{brdf_str} fit for Roughness={roughness}, f0={f0}, V_phi={V_phi}")
        ax.plot_surface(P[0], P[1], P[2], antialiased=False, label=f_str)
        for k, sg in enumerate(sgs, start=MIN_SG):
            y = f(L, sg).reshape(H, W)
            P = L.reshape(3, H, W) * y
            try:
                ax.plot_surface(P[0], P[1], P[2], antialiased=False, label=f"{f_str}: {k} SG(s)")
            except ValueError:
                continue
        ax.legend()
        plt.show()